In [29]:
import os
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
INT_S2_DIR = os.path.join("_intermediate", "stage2")
DATASET_DIR = os.path.join("_data", "plant_pathology")
INDEX = 0

In [7]:
train_data = pd.read_csv(os.path.join(DATASET_DIR, "train.csv"))

In [8]:
train_data

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [26]:
train_data.iloc[198]

image_id             Train_198
healthy                      1
multiple_diseases            0
rust                         0
scab                         0
Name: 198, dtype: object

In [3]:
with open(os.path.join(INT_S2_DIR, "total.pkl"), "rb") as f:
    results = pickle.load(f)

/tmp/ipykernel_2229779/3654069281.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  results = pickle.load(f)


In [4]:
results

{'(SAM + YOLOv8) + (InceptionV3)': {198: {'healthy': True,
   'masks': [{'segmentation': array([[False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            ...,
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False]]),
     'area': 1023324,
     'bbox': [384, 142, 1560, 977],
     'predicted_iou': 1.0320401191711426,
     'point_coords': [[1824.0, 575.859375]],
     'stability_score': 0.990678071975708,
     'crop_box': [0, 0, 2048, 1365],
     'patch': array([[[ 90, 124,  64],
             [ 90, 124,  64],
             [ 90, 124,  64],
             ...,
             [168, 223, 184],
             [168, 222, 185],
             [166, 223, 185]],
     
            [[ 91, 123,  64],
             [ 91, 123,  64],
             [ 91, 123,  64],
  

In [81]:
conv = []
coord_list = []

for i, (name, data) in enumerate(results.items()):
    pd_index = list(data.keys())[INDEX]
    train_data.index
    image_file = train_data.iloc[pd_index]['image_id']
    image_path = os.path.join(DATASET_DIR, "images", f'{image_file}.jpg')
    image = plt.imread(image_path)
    data = data[pd_index]
    for mask in data['masks']:
        box = mask['bbox']
        coords = (box[0] + box[2] / 2, box[1] + box[3] / 2)
        
        if coords not in coord_list:
            conv.append({ 'center': coords, 'probs': [0] * len(results) })
            coord_list.append(coords)
        index = coord_list.index(coords)
        conv[index]['probs'][i] = mask['prob_diseased']
        conv[index]['mask'] = mask['segmentation']

In [82]:
conv

[{'center': (1164.0, 630.5),
  'probs': [np.float32(0.00048843044),
   0,
   np.float32(0.4853645),
   0,
   np.float32(0.30440727),
   0],
  'mask': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]])},
 {'center': (969.0, 746.0),
  'probs': [np.float32(5.1139057e-05),
   0,
   np.float32(0.4853646),
   0,
   np.float32(0.3039151),
   0],
  'mask': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., F

In [105]:
def plot_image_with_masks_and_probabilities(image, masks, regions, probabilities, region_labels=None, bar_height_factor=1.5):
    """
    Create an interactive visualization with image, segmentation masks, and probability bars.
    
    Parameters:
    -----------
    image : numpy.ndarray
        The base image to display
    masks : list of numpy.ndarray
        List of binary masks for each region, same size as image
    regions : list of dicts
        List of region information, each containing:
        {
            'center': (x, y),     # Center coordinates of the region
            'width': float,       # Width of the region for bar plot sizing
            'height': float       # Height of the region for bar plot sizing
        }
    probabilities : list of lists
        List of probability lists for each region
    region_labels : list, optional
        Labels for probability categories
    bar_height_factor : float
        Factor to scale the height of the bars (default: 1.5)
    """
    if region_labels is None:
        region_labels = [f'Class {i+1}' for i in range(len(probabilities[0]))]
    
    # Create the base figure
    fig = go.Figure()
    
    # Add the base image
    fig.add_trace(
        go.Image(
            z=image,
            name='Base Image',
            hovertemplate=None,
            hoverinfo='skip'
        )
    )
    
    # Process masks and add contours
    mask_traces = []
    contour_traces = []
    for i, mask in enumerate(masks):
        # Find contours of the mask
        contours = find_contours(mask, 0.5)
        
        # Add a filled shape for the mask (initially invisible)
        mask_points = np.where(mask)
        mask_trace = go.Scatter(
            x=mask_points[1],
            y=mask_points[0],
            mode='markers',
            marker=dict(
                size=1,
                color=f'rgba({50+i*50}, {100+i*50}, {150+i*50}, 0.3)',
            ),
            name=f'Region {i+1} Mask',
            hoverinfo='skip',
            visible=False,
            showlegend=False
        )
        mask_traces.append(mask_trace)
        
        # Add contours for the region
        for contour in contours:
            contour_trace = go.Scatter(
                x=contour[:, 1],
                y=contour[:, 0],
                mode='lines',
                line=dict(
                    color=f'rgba({50+i*50}, {100+i*50}, {150+i*50}, 1)',
                    width=2
                ),
                name=f'Region {i+1} Contour',
                hoverinfo='skip',
                showlegend=False
            )
            contour_traces.append(contour_trace)
    
    # Add bar plots for each region
    bar_traces = []
    for i, region in enumerate(regions):
        center_x, center_y = region['center']
        width = 150
        height = 150 * bar_height_factor
        
        n_bars = len(probabilities[i])
        bar_width = width / (n_bars * 1.5)
        
        for j, prob in enumerate(probabilities[i]):
            bar_x = center_x - (width/4) + (j * bar_width * 1.2)
            
            # Scale the probability to make bars taller
            scaled_prob = prob * height
            
            bar_trace = go.Bar(
                x=[bar_x],
                y=[scaled_prob],
                width=bar_width,
                name=f'Region {i+1} - {region_labels[j]}',
                marker_color=f'rgba({50+i*50}, {100+i*50}, {150+i*50}, 0.7)',
                text=[f'{prob:.2f}'],
                textposition='auto',
                showlegend=True if i == 0 else False,
                base=[center_y],
                orientation='v',
                hovertemplate=f'Region {i+1}<br>{region_labels[j]}: {prob:.2f}<extra></extra>',
                customdata=[i, j],  # Store region and bar index
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=12,
                    font_family="Arial"
                )
            )
            bar_traces.append(bar_trace)
    
    # Add all traces to the figure
    fig.add_traces(mask_traces + contour_traces + bar_traces)
    
    # Add hover callbacks to highlight the region
    fig.update_traces(
        selector=dict(type='bar'),
        hovertemplate="%{data.name}<br>%{customdata[1]}: %{y:.2f}<extra></extra>",
        hoverinfo='name+text'
    )
    
    fig.update_layout(
        hovermode='closest',
        updatemenus=[{
            'buttons': [
                {
                    'args': [{'visible': [True] * len(fig.data)}],
                    'label': 'Show All',
                    'method': 'restyle'
                },
                {
                    'args': [{'visible': [True if i < len(mask_traces + contour_traces) else False for i in range(len(fig.data))]}],
                    'label': 'Hide Bars',
                    'method': 'restyle'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'y': 1.1,
        }]
    )
    
    # Update layout
    fig.update_layout(
        title="Interactive Region Visualization with Probabilities",
        showlegend=True,
        xaxis=dict(
            range=[0, image.shape[1]],
            showgrid=False,
            zeroline=False,
            visible=False
        ),
        yaxis=dict(
            range=[0, image.shape[0]],
            showgrid=False,
            zeroline=False,
            visible=False,
            scaleanchor="x",
            scaleratio=1
        ),
        barmode='group',
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

In [ ]:
fig = plot_image_with_masks_and_probabilities(
    image=image,
    masks=[item['mask'] for item in conv],
    regions=conv,
    probabilities=[item['probs'] for item in conv],
    region_labels=list(results.keys())
)

fig.show()